In [ ]:
!pip install pvlib

In [ ]:
!pip install ephem

In [ ]:
import logging
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from get_pond_data import PondDataHolder
from evaluate_model_daily import ModelDailyEvaluator

## Prediction of pond dissolved oxygen using weather data

In this notebook dissolved oxygen in aquaculture ponds is predicted using Random Forest model.

### Dataset Description

The dataset contains pond water quality parameters such as dissolved oxygen (DO) and temperature taken twice a day, in the morning and in the evening, secchi disc visibility, and weather data taken from a weather station located in a closest town.

The full description of the raw data can be found in get_pond_data.py module. The data that will remain after dropping useless or correlated columns is the following:

- meas_lag: The lag in seconds between the measurement and sunrise/sunset.
- o2: The level of dissolved oxygen in the pond water.
- temp: The temperature of the water in the pond.
- pressure: The atmospheric pressure recorded on the day of the measurement, in millimeters of mercury (mmHg).
- secchi: The transparency of the water, measured in meters by Secchi disc visibility.
- feed_kg: the amount of fish feed given to a pond.
- sun_elev: sinus of max sun elevation angle for the given date.
- moon_phase: The phase of the moon.
- avg_temp: The average air temperature recorded on the day of the measurement.
- delta_temp: The difference between min and max daily air temperature.
- wind_spd: The wind speed recorded on the day of the measurement, in kilometers per hour.
- wind_dir: The wind direction recorded on the day of the measurement, in degrees.
- precip_mm: The amount of precipitation recorded on the day of the measurement, in millimeters.
- cloud_cov: The percentage of the sky covered by clouds on the day of the measurement.
- moonshine: An indicator that I made by multiplying duration of moonshine by the moon illuminated surface and by max moon elevation (sin)
- days_passed: the number of days passed since first measurement was taken.
- pond_no: The identifier of the pond where the measurement was taken. {'np1': 1, 'np2': 2, 'vp1': 3, 'vp2': 4, 'vp3': 5, 'vp4': 6}

The water measurements were taken at dusk or dawn.
The weather data is provided for every 24-hour interval from www.worldweatheronline.com

NaNs for o2 and Secchi visibility are filled using interpolation in get_pond_data.py module.

### Retrieve the data

The get_pond_data.py module has a PondDataHolder class that takes the csv file containing data for all ponds and creates a dictionary that holds a separate data frame for each pond.

The raw data rows each corresponds to a single measurement, while the data in the PondDataHolder is processed so that every row corresponds to a day and contains both morning and evening measurements.

Let's use the module to load the data.

In [ ]:
logging.basicConfig(level=logging.INFO)

encode_ponds = {'np1': 1, 'np2': 2, 'vp1': 3, 'vp2': 4, 'vp3': 5, 'vp4': 6}

ponds_data = PondDataHolder(ponds_encode=encode_ponds,
                            filename='pond_weather_combined_23.07.2023.csv')

# Some ponds have too few data, or didn't have fish in them, so remove
to_remove = ['lmp', 'zm1', 'vp5']

# This populates the data holding dictionary and drops the useless data
ponds_data.populate_ponds_dict(to_remove)

# Here morning and evening measurements are merged into daily measurements
ponds_data.merge_mor_eve()

# Create target variable that is oxygen in the morning of the next day
# We also let the model know the time at which the measurement will be taken tomorrow.
ponds_data.create_target_var_tmrw_mor()

Let's see what columns we end up having in our dataframes and see if some of them are correlated.

In [ ]:
print(ponds_data.ponds['np1'].columns)

In [ ]:
cols_to_check = ['wind_spd', 'wind_gust', 'cloud_cov', 'humidity', 'dew_pt', 'avg_temp', 'sun_sec','sun_elev']

# Compute the correlation matrix
corr = ponds_data.ponds['np1'][cols_to_check].corr()

# Create a heatmap
plt.figure(figsize=(10, 8))  # You can specify your own figsize here
sns.heatmap(corr, cmap='coolwarm', annot=True, fmt=".2f", linewidths=0.5, linecolor='white', vmin=-1, vmax=1)

plt.title("Correlation Matrix")
plt.show()

### Drop useless columns

Wind speed and wind gust are linearly correlated, so keep the speed only

Humidity and cloud coverage are correlated as well, so drop humidity

Dew_pt is correlated with air temperature

We have delta temp and avg temp for air, so no need for max and min

sun_sec and sun_elev are correlated, so keep only sun_elev

In [ ]:
cols_to_drop = ['weath_code', 'cloud_desc', 'sun_hr', 'uv_idx', 'date',
                'sunrise', 'sunset', 'wind_gust', 'humidity', 'dew_pt',
                'min_temp', 'max_temp', 'sun_sec', 'o2_sat_eve', 'o2_sat_mor']
ponds_data.drop_cols(cols_to_drop)

The columns that are left:

In [ ]:
print(ponds_data.ponds['np1'].columns)

### Lagged features

Lagged features are simply past values of our data. So, a 1-day lagged feature of our temperature data would be yesterday's temperature, a 2-day lag would be the temperature from two days ago, and so on.

Let's create some lagged features with lag 1 and 2. The exclude list contains features which

In [ ]:
exclude = ['days_passed', 'y', 'tmrw_meas_lag_mor', 'pond_no', 'sun_elev']
ponds_data.add_lagged_features(lag=2, exclude=exclude)
print(ponds_data.ponds['np1'].columns)

Here is the Random Forest model. The parameters were adjusted using grid search.

In [ ]:
best_parameters = {
    'n_estimators': 100,
    'max_depth': 20,
    'max_features': 'sqrt',
    'max_leaf_nodes': None,
    'min_impurity_decrease': 0.0,
    'min_samples_leaf': 2,
    'min_samples_split': 2,
    'min_weight_fraction_leaf': 0.0,
    'n_jobs': -1
}

rf = RandomForestRegressor(**best_parameters)

### Model in action

Most of the code is hidden under the hood in the evaluate_model_daily.py module. In brief, all it does is trains the model on a window of several days, then predicts DO in the morning for the next day. It trains on data for all ponds combined. Then the window is extenden by one day forward and the process is repeated.

Before this, the most important features are selected using RFE (recursive feature elimination) from sklearn.feature_selection.

The importance of features for the model trained on the max window is plotted below. 

In [ ]:
# It takes data as first argument and the dictionary with pond names encoding as the second
evaluator = ModelDailyEvaluator(ponds_data.ponds, encode_ponds)

# The first argument is our Random Forest model, the second is the maximum number of features
results = evaluator.evaluate_all_ponds_daily(rf, n_feat_rfe=20)

### Factors affecting dissolved oxygen

Besides dissolved oxygen and water temperature for past days, the most important features were:

- Sun elevation
- The number of days passed
- Moonshine
- Atmospheric pressure
- Pond number
- Cloud coverage
- Measurement lag in the evening (number of seconds between sunset and the measurement)
- Amount of fish feed given
- Time at which to predict oxygen the next day (tomorrow morning lag)

### Results

The graphs show dissolved oxygen for each pond in the morning. One line is for the predicted data and the other is for the real.

The results are:

pond: np1, RMSE: 1.21, R2: 0.44

pond: np2, RMSE: 1.07, R2: 0.31

pond: vp1, RMSE: 2.15, R2: 0.42

pond: vp2, RMSE: 1.54, R2: 0.36

pond: vp3, RMSE: 1.39, R2: 0.58

pond: vp4, RMSE: 1.24, R2: 0.69

The model predicts ponds vp3 and vp4 data best, because oxygen diurnal oscillations are narrow and stable for these ponds.

SARIMA model's results were better for stable ponds (vp3 and vp4) and worse for ponds with less diurnal seasonality:

pond: np1, RMSE: 1.744, R2: 0.26

pond: np2, RMSE: 1.512, R2: 0.12

pond: vp1, RMSE: 2.091, R2: 0.56

pond: vp2, RMSE: 1.972, R2: 0.40

pond: vp3, RMSE: 0.779, R2: 0.75

pond: vp4, RMSE: 0.992, R2: 0.74

As a reference a previous research titled "Machine learning for manually-measured water quality prediction in fish farming" published in PLoS One achieved an RMSE of 1.1787 and R2 of 0.62.
https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0256380

In [ ]:
evaluator.plot_results(show_pic=True)